# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [3]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

100.1%Extracting ./mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw
113.5%Extracting ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist/MNIST/raw
100.4%Extracting ./mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist/MNIST/raw
180.4%Extracting ./mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist/MNIST/raw
Processing...
Done!
/home/ktclyf/miniconda3/envs/tmp/lib/python3.6/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Then, we define the model, object function and optimizer that we use to classify.

In [14]:
class SimpleNet(nn.Module):
# TODO:define model

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=(5, 5))
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        self.conv2 = nn.Conv2d(6, 16, kernel_size=(3, 3))
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        self.conv3 = nn.Conv2d(16, 120, kernel_size=(5, 5))

        self.f4 = nn.Linear(120, 84)

        self.f5 = nn.Linear(84, 10)
        self.sig5 = nn.LogSoftmax(dim=-1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        y = F.relu(self.conv2(x))
        y = self.pool2(y)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x += y
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.f4(x))
        x = self.f5(x)

        x = self.sig5(x)

        return x

    
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)

Next, we can start to train and evaluate!

In [17]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        optimizer.zero_grad()
        output = model(images)

        loss = criterion(output, labels)        
        
        loss.backward()
        optimizer.step()
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
def test(model,dataloader,dataset):
    model.eval()
    total_correct = 0
    avg_loss = 0.0
    for images, labels in tqdm(dataloader):
        
        output = model(images)
        avg_loss += criterion(output, labels).sum()
        pred = output.detach().max(1)[1]
        total_correct += pred.eq(labels.view_as(pred)).sum()

    avg_loss /= len(dataset)
    acc = float(total_correct) / len(dataset)
    
    return acc
    
train_acc = test(model, train_loader, train_dataset)
test_acc = test(model, test_loader, test_dataset)


100%|██████████| 78/78 [00:01<00:00, 53.43it/s]


#### Q5:
Please print the training and testing accuracy.

In [18]:
print('Training accuracy: %0.2f%%' % (train_acc*100))
print('Testing accuracy: %0.2f%%' % (test_acc*100))

Training accuracy: 99.57%
Testing accuracy: 98.69%
